# Palmer Penguins Predictive Models

## EDA

In [ ]:
import pandas as pd
import numpy as np
from palmerpenguins import load_penguins

# Load the penguins dataset
penguins = load_penguins()

# Convert to a DataFrame
df = pd.DataFrame(penguins)

# Display the first few rows of the DataFrame
print(df.head())

In [ ]:
# Use ydata-profiling to generate a report
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Penguins Dataset Profiling Report", explorative=True)
# Save the report to an HTML file
profile.to_file("penguins_profiling_report.html")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

species_island_counts = df.groupby(['species', 'island']).size().unstack()
species_island_counts.plot(kind='bar', stacked=True)
plt.ylabel('Count')
plt.title('Penguin Species Distribution by Island')
plt.show()

## Training models

### Import Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

### Load DataSet

In [ ]:
penguins = load_penguins()
penguins = penguins.dropna(subset=['species'])

# Separate features and target
X = penguins.drop(columns='species')
y = penguins['species']

# Separate features and target
X = penguins.drop(columns='species')
y = penguins['species']

# Identify column types
numeric_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# categorical_features = ['sex']
categorical_features = ['island', 'sex']

# Preprocessing for numeric data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### Create Models

In [ ]:
models = {
    "logistic_regression": LogisticRegression(max_iter=1000),
    "random_forest": RandomForestClassifier(random_state=42),
    "svc": SVC(kernel='rbf', probability=True, random_state=42)
}

### Train, evaluate and save

In [ ]:
results = {}
for name, model in models.items():
    clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Save the full pipeline (preprocessing + model)
    model_path = f"../models/{name}.joblib"
    joblib.dump(clf, model_path)
    print(f"✅ Saved model to {model_path}")

print("\nModel comparison:", results)

## Training New Models

In [ ]:
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

new_models = {  
    "catboost": CatBoostClassifier(random_state=42, verbose=False),
    "adaboost": AdaBoostClassifier(n_estimators=100, random_state=42)
}

In [ ]:
new_results = {}
for name, model in new_models.items():
    clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    new_results[name] = acc
    
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Save the full pipeline (preprocessing + model)
    model_path = f"../models/{name}.joblib"
    joblib.dump(clf, model_path)
    print(f"✅ Saved model to {model_path}")

print("\nModel comparison:", new_results)